# Lab 3 — The Specificity Spectrum

**Module reference:** [Module 2, §2.1](https://github.com/kunalsuri/prompt-engineering-playbook/blob/main/learn/02-core-principles.md)

This lab demonstrates how increasing prompt specificity improves output quality. It sends three versions of the same request — **vague**, **moderate**, and **highly specific** — then scores the outputs using an **LLM-as-Judge** evaluation.

---

### Free API Options
| Provider | Free Tier | Sign Up |
|---|---|---|
| **Google Gemini** (recommended) | 15 RPM, 1M tokens/day | [aistudio.google.com/apikey](https://aistudio.google.com/apikey) |
| **Groq** | 30 RPM, 14.4K tokens/min | [console.groq.com](https://console.groq.com) |
| **OpenAI** (paid) | Pay-per-token | [platform.openai.com](https://platform.openai.com/api-keys) |

In [ ]:
#@title 🔧 Setup — Run this cell first
!pip install -q openai

import getpass, os

print("Choose your LLM provider (all work with this lab):")
print("  1. Google Gemini (FREE — recommended)")
print("  2. Groq (FREE)")
print("  3. OpenAI (paid)")
choice = input("\nEnter 1, 2, or 3: ").strip()

if choice == "1":
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")
    BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
    API_KEY = os.environ["GOOGLE_API_KEY"]
    MODEL = "gemini-2.0-flash"
elif choice == "2":
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")
    BASE_URL = "https://api.groq.com/openai/v1"
    API_KEY = os.environ["GROQ_API_KEY"]
    MODEL = "llama-3.1-8b-instant"
else:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
    BASE_URL = None
    API_KEY = os.environ["OPENAI_API_KEY"]
    MODEL = "gpt-4o-mini"

from openai import OpenAI
client_kwargs = {"api_key": API_KEY}
if BASE_URL:
    client_kwargs["base_url"] = BASE_URL
client = OpenAI(**client_kwargs)

def complete(prompt, *, system="", temperature=0.3, max_tokens=1024):
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})
    response = client.chat.completions.create(
        model=MODEL, messages=messages,
        temperature=temperature, max_tokens=max_tokens
    )
    return response.choices[0].message.content or ""

print(f"\n✅ Connected to {MODEL}")

## Three Prompt Variants

In [ ]:
SYSTEM = "You are a helpful writing assistant."

PROMPTS = {
    "vague": "Write something about Python testing.",
    "moderate": (
        "Write a guide about testing in Python. Cover unit tests and "
        "integration tests. Include code examples."
    ),
    "specific": (
        "Write a concise guide (300-400 words) on Python testing best "
        "practices for a mid-level developer. Structure the guide with "
        "these sections:\n"
        "1. **Unit Tests** — explain the AAA pattern (Arrange, Act, Assert) "
        "with a pytest example testing a `calculate_discount(price, percent)` "
        "function.\n"
        "2. **Integration Tests** — show a pytest example that tests a "
        "FastAPI endpoint using `TestClient`.\n"
        "3. **Key Principles** — list 3 bullet-point testing principles "
        "(e.g., test isolation, determinism, meaningful names).\n\n"
        "Use fenced Python code blocks. End with a one-sentence summary."
    ),
}

print("Prompts defined at three specificity levels.")
for level, prompt in PROMPTS.items():
    print(f"  {level:>10}: {prompt[:60]}...")

## Generate Outputs

In [ ]:
outputs = {}
for level, prompt in PROMPTS.items():
    print(f"Generating {level} output...")
    outputs[level] = complete(prompt, system=SYSTEM)
    print(f"  → {len(outputs[level])} chars, {len(outputs[level].split())} words\n")

print("✅ All outputs generated.")

## LLM-as-Judge Evaluation

In [ ]:
import json

EVAL_SYSTEM = "You are a strict technical writing evaluator. Return ONLY valid JSON."

EVAL_TEMPLATE = (
    "Evaluate the following technical guide on a 1-5 scale for each criterion.\n"
    "Return ONLY a JSON object with these keys:\n"
    "- structure: Does it have clear sections/headers? (1=none, 5=excellent)\n"
    "- detail: Are explanations specific and actionable? (1=vague, 5=very specific)\n"
    "- code_quality: Are code examples present, correct, and useful? (1=none/poor, 5=excellent)\n"
    "- completeness: Does it cover the topic adequately? (1=minimal, 5=comprehensive)\n"
    "- overall: Overall quality (1=poor, 5=excellent)\n\n"
    "Guide to evaluate:\n---\n{guide}\n---\n\nJSON:"
)

scores = {}
for level, output in outputs.items():
    print(f"Evaluating {level} output...")
    raw = complete(EVAL_TEMPLATE.format(guide=output[:2000]), system=EVAL_SYSTEM, temperature=0.0)
    try:
        start = raw.find("{")
        end = raw.rfind("}") + 1
        scores[level] = json.loads(raw[start:end]) if start >= 0 else {}
    except (json.JSONDecodeError, ValueError):
        scores[level] = {}

print("\n✅ Evaluation complete!")

## Results

In [ ]:
import pandas as pd

criteria = ["structure", "detail", "code_quality", "completeness", "overall"]
rows = []
for c in criteria:
    rows.append({
        "Criterion": c.replace("_", " ").title(),
        "Vague": scores.get("vague", {}).get(c, "?"),
        "Moderate": scores.get("moderate", {}).get(c, "?"),
        "Specific": scores.get("specific", {}).get(c, "?"),
    })
display(pd.DataFrame(rows))

print("\n📝 Takeaway: More specific prompts consistently produce higher-quality,")
print("more structured output. The 'specific' prompt constrains word count, defines")
print("exact sections, names functions, and specifies the audience.")

## Output Previews

Expand the cells below to see the actual generated output at each specificity level.

In [ ]:
#@title Vague Output (click to expand)
print(outputs["vague"][:800])
if len(outputs["vague"]) > 800:
    print("\n... [truncated]")

In [ ]:
#@title Moderate Output (click to expand)
print(outputs["moderate"][:800])
if len(outputs["moderate"]) > 800:
    print("\n... [truncated]")

In [ ]:
#@title Specific Output (click to expand)
print(outputs["specific"][:800])
if len(outputs["specific"]) > 800:
    print("\n... [truncated]")